In [ ]:
# 모듈 불러오기
import time
import board
import busio
import adafruit_ads1x15.ads1115 as ADS
from adafruit_ads1x15.analog_in import AnalogIn
import pandas as pd
import numpy as np
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from datetime import datetime
import os
import sys

# 알림음 (비프음) 설정
import os
def play_beep():
    os.system('aplay /usr/share/sounds/alsa/Front_Center.wav &')  # 또는 beep 명령어

In [ ]:
i2c = busio.I2C(board.SCL, board.SDA)
ads = ADS.ADS1115(i2c)
ads.gain = 1

voltage_ch = AnalogIn(ads, ADS.P0)
current_ch = AnalogIn(ads, ADS.P1)

In [ ]:
# 보정 파라미터 
ZMPT_offset = 1.65  # 중심값 (ZMPT101B 기준)
ZMPT_scale = 57.14  # 1.65V → 약 220V일 때, 실험해서 보정 필요

ACS712_offset = 2.5      # 0A일 때 기준 전압
ACS712_sensitivity = 0.1 # 20A 버전: 100mV/A

In [ ]:
# 모델 로딩
model = joblib.load('./model/random_forest_model.joblib')
scaler = joblib.load('./model/scaler_rf.joblib')

In [ ]:
# 후처리 함수
def postprocess_anomalies(y_bin, min_consecutive=8):
    y_post = np.zeros_like(y_bin)
    count = 0
    for i, val in enumerate(y_bin):
        if val == 1:
            count += 1
        else:
            if count >= min_consecutive:
                y_post[i - count:i] = 1
            count = 0
    if count >= min_consecutive:
        y_post[len(y_bin)-count:] = 1
    return y_post

In [ ]:
# 파생 피처 함수 정의
def add_features(df):
    df['voltage_diff'] = df['voltage'].diff().fillna(0).abs()
    df['current_diff'] = df['current'].diff().fillna(0).abs()
    df['voltage_ma'] = df['voltage'].rolling(5).mean().bfill()
    df['current_ma'] = df['current'].rolling(5).mean().bfill()
    df['power'] = df['voltage'] * df['current']
    df['power_diff'] = df['power'].diff().fillna(0).abs()
    return df

In [ ]:
# 센서 데이터 읽기 함수
def read_sensor_data():
    v_raw = voltage_ch.voltage
    c_raw = current_ch.voltage

    voltage = (v_raw - ZMPT_offset) * ZMPT_scale
    current = (c_raw - ACS712_offset) / ACS712_sensitivity

    return voltage, current

In [ ]:
# 실시간 예측

buffer = []
log_list = []

print("🔁 실시간 아크 감지 시작")

try:
    while True:
        voltage, current = read_sensor_data()
        timestamp = datetime.now().strftime('%H:%M:%S.%f')[:-3]
        buffer.append({'voltage': voltage, 'current': current, 'time': timestamp})

        if len(buffer) >= 50:
            df = pd.DataFrame(buffer[-50:])
            df = add_features(df)
            X = df[['voltage', 'current', 'voltage_diff', 'current_diff',
                    'voltage_ma', 'current_ma', 'power', 'power_diff']].fillna(0).values
            X_scaled = scaler.transform(X)
            y_pred = model.predict(X_scaled)
            y_post = postprocess_anomalies(y_pred)

            if y_post[-1] == 1:
                print(f"⚠️ 아크 감지됨! ⏱️ {timestamp}")
                log = {'time': timestamp, 'voltage': voltage, 'current': current, 'prediction': 1}
                log_list.append(log)
                play_beep()
                time.sleep(30) # 아크 발견시 멈추기

        time.sleep(0.0001)

except KeyboardInterrupt:
    print("🛑 종료되었습니다.")
    if log_list:
        log_df = pd.DataFrame(log_list)
        log_df.to_csv('arc_log.csv', index=False)
        print("✅ arc_log.csv 파일로 저장 완료")
    else:
        print("ℹ️ 감지된 아크 없음.")